## Combining dataframes and model fitting


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import csv files
train = pd.read_csv(
    'train.csv', 
    low_memory=False, 
    nrows=10**6, 
    dtype={
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int8', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
    }
)
question = pd.read_csv('questions.csv')
lecture = pd.read_csv('lectures.csv')
sample_test = pd.read_csv('example_test.csv')

In [3]:
train = train[train.content_type_id ==0]
train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...,...
999995,999995,26482248,20949024,8803,0,29,1,1,14000.0,True
999996,999996,26516686,20949024,4664,0,30,3,1,17000.0,True
999997,999997,26537967,20949024,4108,0,31,1,0,18000.0,True
999998,999998,26590240,20949024,5014,0,32,3,0,6000.0,True


In [4]:
train_exp = train[['user_id', 'timestamp',
                  'content_id', 'answered_correctly', 
                  'prior_question_elapsed_time', 
                   'prior_question_had_explanation']]
train_exp.head()

,user_id,timestamp,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,115,0,5692,1,NaN,<NA>
1,115,56943,5716,1,37000.0,False
2,115,118363,128,1,55000.0,False
3,115,131167,7860,1,19000.0,False
4,115,137965,7922,1,11000.0,False


In [5]:
# most actions top 20 user
train_15 = train['user_id'].value_counts().reset_index()

In [22]:
train_15.columns = ('top20_id', 'count')
train_15 = train_15.sort_values(by = ['count'], ascending = False).iloc[:15]
train_15 

,top20_id,count
0,7171715,10797
1,1283420,7476
2,18122922,7413
3,9418512,7261
4,4421282,6960
5,3838215,6763
6,11610328,6540
7,15441391,6461
8,24418,6283
9,19489973,6101


In [23]:
top15 = train_15.merge(train_exp, left_on='top20_id', right_on='user_id')
top15 = top15[['top20_id','count', 'timestamp',
                  'content_id', 'answered_correctly', 
                  'prior_question_elapsed_time', 
                   'prior_question_had_explanation']]
top15.columns = ('user_id', 'action_count', 'time', 'content_id', 'asw_correct', 'prior_time', 'prior_explain' )
top15

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain
0,7171715,10797,0,4318,1,NaN,<NA>
1,7171715,10797,4274,5112,1,2000.0,False
2,7171715,10797,7649,6143,0,1000.0,False
3,7171715,10797,11134,5070,0,1000.0,False
4,7171715,10797,14275,4554,1,1000.0,False
...,...,...,...,...,...,...,...
99905,15622198,5514,3817887268,3383,0,38666.0,True
99906,15622198,5514,3817887268,3381,1,38666.0,True
99907,15622198,5514,3818314019,2739,1,28333.0,True
99908,15622198,5514,3818314019,2738,1,28333.0,True


In [27]:
# get aggregated data for each user_id
std_accu = top15[top15.asw_correct != -1].groupby('user_id')['asw_correct'].mean().reset_index()
# time_total = train[train.answered_correctly != -1].groupby('user_id')['prior_question_elapsed_time'].max().reset_index()
std_accu.columns = ('user_id', 'std_accu')
std_accu

,user_id,std_accu
0,24418,0.690275
1,1283420,0.828785
2,1440282,0.769395
3,3838215,0.757060
4,4421282,0.757328
5,4742725,0.648727
6,6610089,0.698692
7,7171715,0.261462
8,9418512,0.608869
9,10223055,0.591793


In [28]:
# join on user_id

top15 = top15.join(std_accu.set_index('user_id'), on= 'user_id')
top15.head()

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain,std_accu
0,7171715,10797,0,4318,1,NaN,<NA>,0.261462
1,7171715,10797,4274,5112,1,2000.0,False,0.261462
2,7171715,10797,7649,6143,0,1000.0,False,0.261462
3,7171715,10797,11134,5070,0,1000.0,False,0.261462
4,7171715,10797,14275,4554,1,1000.0,False,0.261462


In [29]:
question.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [30]:
# get accuracy rate for each question
train_q = train[train.content_type_id == 0]
# train_q['answered_correctly'].min()
accu_q = train_q.groupby('content_id').answered_correctly.mean().reset_index()

accu_q.columns = ('content_id', 'accu_q')
accu_q

,content_id,accu_q
0,0,0.863014
1,1,0.927273
2,2,0.560811
3,3,0.798995
4,4,0.602606
...,...,...
13071,13518,0.750000
13072,13519,0.555556
13073,13520,0.700000
13074,13521,0.857143


In [31]:
top15

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain,std_accu
0,7171715,10797,0,4318,1,NaN,<NA>,0.261462
1,7171715,10797,4274,5112,1,2000.0,False,0.261462
2,7171715,10797,7649,6143,0,1000.0,False,0.261462
3,7171715,10797,11134,5070,0,1000.0,False,0.261462
4,7171715,10797,14275,4554,1,1000.0,False,0.261462
...,...,...,...,...,...,...,...,...
99905,15622198,5514,3817887268,3383,0,38666.0,True,0.702031
99906,15622198,5514,3817887268,3381,1,38666.0,True,0.702031
99907,15622198,5514,3818314019,2739,1,28333.0,True,0.702031
99908,15622198,5514,3818314019,2738,1,28333.0,True,0.702031


In [32]:
top15 = top15.join(accu_q.set_index('content_id'), on= 'content_id')

In [33]:
top15

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain,std_accu,accu_q
0,7171715,10797,0,4318,1,NaN,<NA>,0.261462,0.764706
1,7171715,10797,4274,5112,1,2000.0,False,0.261462,0.800000
2,7171715,10797,7649,6143,0,1000.0,False,0.261462,0.360424
3,7171715,10797,11134,5070,0,1000.0,False,0.261462,0.353448
4,7171715,10797,14275,4554,1,1000.0,False,0.261462,0.674797
...,...,...,...,...,...,...,...,...,...
99905,15622198,5514,3817887268,3383,0,38666.0,True,0.702031,0.800000
99906,15622198,5514,3817887268,3381,1,38666.0,True,0.702031,0.981818
99907,15622198,5514,3818314019,2739,1,28333.0,True,0.702031,0.742857
99908,15622198,5514,3818314019,2738,1,28333.0,True,0.702031,0.600000


In [35]:
question = question[ ['question_id', 'part']]
question

,question_id,part
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
13518,13518,5
13519,13519,5
13520,13520,5
13521,13521,5


In [36]:
top15 = top15.join(question.set_index('question_id'), on = 'content_id')
top15

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain,std_accu,accu_q,part
0,7171715,10797,0,4318,1,NaN,<NA>,0.261462,0.764706,5
1,7171715,10797,4274,5112,1,2000.0,False,0.261462,0.800000,5
2,7171715,10797,7649,6143,0,1000.0,False,0.261462,0.360424,5
3,7171715,10797,11134,5070,0,1000.0,False,0.261462,0.353448,5
4,7171715,10797,14275,4554,1,1000.0,False,0.261462,0.674797,5
...,...,...,...,...,...,...,...,...,...,...
99905,15622198,5514,3817887268,3383,0,38666.0,True,0.702031,0.800000,4
99906,15622198,5514,3817887268,3381,1,38666.0,True,0.702031,0.981818,4
99907,15622198,5514,3818314019,2739,1,28333.0,True,0.702031,0.742857,4
99908,15622198,5514,3818314019,2738,1,28333.0,True,0.702031,0.600000,4


In [37]:
top15.isnull().describe()

,user_id,action_count,time,content_id,asw_correct,prior_time,prior_explain,std_accu,accu_q,part
count,99910,99910,99910,99910,99910,99910,99910,99910,99910,99910
unique,1,1,1,1,1,2,2,1,1,1
top,False,False,False,False,False,False,False,False,False,False
freq,99910,99910,99910,99910,99910,99895,99895,99910,99910,99910


In [50]:
# train_sample = train_exp.sample(n=50000, random_state=1)
# train_sample

,user_id,timestamp,content_id,answered_correctly,prior_question_elapsed_time,prior_question_elapsed_time,std_accu,time_total,action_total
543389,11038066,14199666914,1253,1,18000.0,18000.0,0.702160,142000.0,2592
55899,1283420,6287796324,10655,1,20000.0,20000.0,0.828785,246000.0,7476
105089,2223671,491002068,6594,1,31750.0,31750.0,0.736301,79000.0,292
679880,13832598,1559002610,6004,0,31000.0,31000.0,0.550000,125000.0,380
782731,15956793,18808024091,258,1,19000.0,19000.0,0.672868,300000.0,645
...,...,...,...,...,...,...,...,...,...
553486,11412125,426685315,9704,1,39000.0,39000.0,0.663617,279000.0,3826
350138,6812883,9996944595,4927,1,37000.0,37000.0,0.489549,296000.0,2727
698978,14442037,6545038401,383,1,18000.0,18000.0,0.551036,199000.0,1352
486011,9418512,14831215922,7449,1,40666.0,40666.0,0.608869,300000.0,7261


In [53]:
# export sampled data
# train_sample.to_csv('train_sample.csv', index = False)

In [38]:
top15.to_csv('top15.csv', index= False)